In [ ]:
import random

class NakliLLM:

    def __init__(self):
        print('LLM created')

    def predict(self, prompt):
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]
        return {'response': random.choice(response_list)}

In [ ]:
client = NakliLLM()
client.predict('who r u')

In [ ]:
class NakliPromptTemplate:
    def __init__(self, template, input_variables):
        self.template = template
        self.input_variables = input_variables

    def format(self, input_dict):
        return self.template.format(**input_dict)

In [ ]:
template = NakliPromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [ ]:
prompt = template.format({'length':'short','topic':'india'})

In [ ]:
llm = NakliLLM()
llm.predict(prompt)


### making chain directly

In [ ]:
class NakliLLMChain:
    def __init__(self, llm, prompt):
        self.llm = llm
        self.prompt = prompt

    def run(self, input_dict):
        final_prompt = self.prompt.format(input_dict)
        result = self.llm.predict(final_prompt)
        return result['response']


In [ ]:
template = NakliPromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [ ]:
llm = NakliLLM()


In [ ]:
chain = NakliLLMChain(llm, template)


In [ ]:
chain.run({'length':'short', 'topic': 'india'})


# Create runnable (to fix reproducibility) for chain

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class Runnable(ABC):

    @abstractmethod
    def invoke(input_data):
        pass

In [ ]:
import random

class NakliLLM(Runnable):

    def __init__(self):
        print('LLM created')

    def invoke(self, prompt):
        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]

        return {'response': random.choice(response_list)}


    def predict(self, prompt):  # the older method is kept for now

        response_list = [
            'Delhi is the capital of India',
            'IPL is a cricket league',
            'AI stands for Artificial Intelligence'
        ]

        return {'response': random.choice(response_list)}

In [ ]:
class NakliPromptTemplate(Runnable):

    def __init__(self, template, input_variables):
        self.template = template
        self.input_variables = input_variables

    def invoke(self, input_dict):
        return self.template.format(**input_dict)

    def format(self, input_dict):   # the older method is kept for now
        return self.template.format(**input_dict)

In [ ]:
class NakliStrOutputParser(Runnable):

    def __init__(self):
        pass

    def invoke(self, input_data):
        return input_data['response']

In [ ]:
class RunnableConnector(Runnable):

    def __init__(self, runnable_list):
        self.runnable_list = runnable_list

    def invoke(self, input_data):

        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)

        return input_data

In [ ]:
template = NakliPromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [ ]:
llm = NakliLLM()

In [ ]:
parser = NakliStrOutputParser()

In [ ]:
chain = RunnableConnector([template, llm, parser])

In [ ]:
chain.invoke({'length':'long', 'topic':'india'})

## Join multiple *chains* to 1 chain

In [ ]:
template1 = NakliPromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

In [ ]:
template2 = NakliPromptTemplate(
    template='Explain the following joke {response}',
    input_variables=['response']
)

In [ ]:
llm = NakliLLM()


In [ ]:
parser = NakliStrOutputParser()

In [ ]:
chain1 = RunnableConnector([template1, llm])

In [ ]:
chain2 = RunnableConnector([template2, llm, parser])

In [ ]:
final_chain = RunnableConnector([chain1, chain2])

In [ ]:
final_chain.invoke({'topic':'cricket'})